# Exploration, testing, visualization

## Step 2: Load and visualize point clouds

This notebook demonstrates how to load and visualize point clouds using Open3D.


In [ ]:
import sys
from pathlib import Path

# Add backend to path
sys.path.append(str(Path().resolve().parent))

from backend.data.datasets import load_point_cloud, normalize_point_cloud, get_point_cloud_info
from backend.visualization.visualize_o3d import show_point_cloud
import open3d as o3d
import numpy as np


### Example 1: Create and visualize a synthetic point cloud


In [ ]:
# Create a simple sphere point cloud for testing
mesh = o3d.geometry.TriangleMesh.create_sphere(radius=1.0, resolution=30)
pcd = mesh.sample_points_uniformly(number_of_points=2000)

# Display info
info = get_point_cloud_info(pcd)
print(f"Point cloud info: {info}")

# Normalize
pcd_normalized = normalize_point_cloud(pcd)

# Visualize
show_point_cloud(pcd_normalized, window_name="Synthetic Sphere")


### Example 2: Load a point cloud from file

To load a .pcd or .ply file, uncomment and modify the path below:


In [ ]:
# Uncomment and set your file path:
# file_path = "path/to/your/pointcloud.pcd"  # or .ply
# pcd = load_point_cloud(file_path)
# pcd_normalized = normalize_point_cloud(pcd)
# show_point_cloud(pcd_normalized)


## Step 3: Preprocessing - Downsampling, Normalization, Partial Views

This section demonstrates preprocessing operations on point clouds.


In [ ]:
from backend.data.preprocessing import (
    downsample_voxel,
    normalize_point_cloud,
    create_partial_point_cloud,
    add_gaussian_noise,
    mask_points_by_angle
)


### Example 1: Downsampling with Voxel Grid


In [ ]:
# Create a dense point cloud
mesh = o3d.geometry.TriangleMesh.create_sphere(radius=1.0, resolution=50)
pcd_original = mesh.sample_points_uniformly(number_of_points=10000)

print(f"Original: {len(pcd_original.points)} points")

# Downsample
pcd_downsampled = downsample_voxel(pcd_original, voxel_size=0.1)
print(f"Downsampled: {len(pcd_downsampled.points)} points")

# Visualize comparison
from backend.visualization.visualize_o3d import show_point_clouds
show_point_clouds([pcd_original, pcd_downsampled], window_name="Original vs Downsampled")


### Example 2: Normalization Methods


In [ ]:
# Create a point cloud with arbitrary scale and position
mesh = o3d.geometry.TriangleMesh.create_box(width=5, height=3, depth=2)
pcd_original = mesh.sample_points_uniformly(number_of_points=2000)

# Apply different normalization methods
pcd_unit_sphere = normalize_point_cloud(pcd_original, method='unit_sphere')
pcd_centered = normalize_point_cloud(pcd_original, method='centered')
pcd_zero_one = normalize_point_cloud(pcd_original, method='zero_one')

print("Normalization methods applied:")
print(f"  Original: {np.asarray(pcd_original.points)[0]}")
print(f"  Unit sphere: {np.asarray(pcd_unit_sphere.points)[0]}")
print(f"  Centered: {np.asarray(pcd_centered.points)[0]}")
print(f"  Zero-one: {np.asarray(pcd_zero_one.points)[0]}")

# Visualize
show_point_clouds([pcd_original, pcd_unit_sphere], window_name="Original vs Normalized")


### Example 3: Create Partial Point Cloud (Mask Side)


In [ ]:
# Create a complete point cloud
mesh = o3d.geometry.TriangleMesh.create_sphere(radius=1.0, resolution=30)
pcd_complete = mesh.sample_points_uniformly(number_of_points=3000)

# Create partial view (mask one side)
pcd_partial = create_partial_point_cloud(
    pcd_complete,
    method='mask_side',
    direction=[1, 0, 0],  # Mask points on +X side
    mask_ratio=0.5
)

print(f"Complete: {len(pcd_complete.points)} points")
print(f"Partial: {len(pcd_partial.points)} points")

# Visualize
show_point_clouds([pcd_complete, pcd_partial], window_name="Complete vs Partial")


### Example 4: Simulate Sensor Field of View (Angle > 45°)


In [ ]:
# Create a point cloud
mesh = o3d.geometry.TriangleMesh.create_sphere(radius=1.0, resolution=30)
pcd_complete = mesh.sample_points_uniformly(number_of_points=3000)

# Method 1: Using create_partial_point_cloud with angle masking
pcd_angle_masked = create_partial_point_cloud(
    pcd_complete,
    method='mask_angle',
    direction=[1, 0, 0],  # Sensor looking along +X
    max_angle=45.0  # Keep only points within 45° field of view
)

# Method 2: Using mask_points_by_angle (same result)
pcd_angle_masked2 = mask_points_by_angle(
    pcd_complete,
    view_direction=[1, 0, 0],
    max_angle=45.0
)

print(f"Complete: {len(pcd_complete.points)} points")
print(f"Angle masked (method 1): {len(pcd_angle_masked.points)} points")
print(f"Angle masked (method 2): {len(pcd_angle_masked2.points)} points")

# Visualize
show_point_clouds([pcd_complete, pcd_angle_masked], window_name="Complete vs Angle Masked")


### Example 5: Add Gaussian Noise


In [ ]:
# Create a clean point cloud
mesh = o3d.geometry.TriangleMesh.create_sphere(radius=1.0, resolution=30)
pcd_clean = mesh.sample_points_uniformly(number_of_points=2000)

# Add Gaussian noise
pcd_noisy = add_gaussian_noise(pcd_clean, std=0.02, relative=True)

print(f"Clean: {len(pcd_clean.points)} points")
print(f"Noisy: {len(pcd_noisy.points)} points")

# Visualize
show_point_clouds([pcd_clean, pcd_noisy], window_name="Clean vs Noisy")


### Example 6: Complete Preprocessing Pipeline


In [ ]:
# Complete preprocessing pipeline
mesh = o3d.geometry.TriangleMesh.create_sphere(radius=1.0, resolution=40)
pcd_original = mesh.sample_points_uniformly(number_of_points=5000)

print("Original:", len(pcd_original.points), "points")

# Step 1: Downsample
pcd = downsample_voxel(pcd_original, voxel_size=0.05)
print("After downsampling:", len(pcd.points), "points")

# Step 2: Create partial view (simulate sensor)
pcd = create_partial_point_cloud(pcd, method='mask_angle', direction=[1, 0, 0], max_angle=45.0)
print("After angle masking:", len(pcd.points), "points")

# Step 3: Add noise (simulate sensor noise)
pcd = add_gaussian_noise(pcd, std=0.01, relative=True)
print("After noise:", len(pcd.points), "points")

# Step 4: Normalize
pcd = normalize_point_cloud(pcd, method='unit_sphere')
print("After normalization:", len(pcd.points), "points")

# Visualize before and after
show_point_clouds([pcd_original, pcd], window_name="Before vs After Preprocessing")
